<h1>DSCI-552 Homework 3</h1>
<br>
<font size="4">Morgan Gautho | GitHub: morgangauth | 3944-8632-72</font>
    

In [1]:
import pandas as pd
import numpy as np
import glob
import os
from scipy.stats import bootstrap


import warnings
warnings.filterwarnings('ignore')

## 1. Time Series Classification Part 1: Feature Creation/Extraction



### 1. (a) Download the AReM data from: https://archive.ics.uci.edu/ml/datasets/Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\%29 .

In [2]:
folders = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']
colNames = ['# Columns: time','avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
parentFolder = "../data/AReM/"

data = pd.DataFrame()

for f in folders:
    # open folder, 
    folderFiles = os.listdir(parentFolder + f)
    idx = 1;
    for c in folderFiles:
        filePath = parentFolder + f + "/" + c
        if((f == folders[1]) & (c == 'dataset4.csv')):
            df = pd.read_csv(filePath, skiprows = 5, sep = ' ', names = colNames, usecols = range(7))
        else:
            df = pd.read_csv(filePath, header = 4, usecols = colNames)
        df['folder'] = f
        df['dataset'] = idx
        if 'bending' in f:
            df['isBending'] = 1
        else:
            df['isBending'] = 0
        idx += 1
        data = pd.concat([data, df], ignore_index=True, axis=0)
    
print(data.head())
            
        



   # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                0      39.25       0.43      22.75       0.43      33.75   
1              250      39.25       0.43      23.00       0.00      33.00   
2              500      39.25       0.43      23.25       0.43      33.00   
3              750      39.50       0.50      23.00       0.71      33.00   
4             1000      39.50       0.50      24.00       0.00      33.00   

   var_rss23    folder  dataset  isBending  
0        1.3  bending1        1          1  
1        0.0  bending1        1          1  
2        0.0  bending1        1          1  
3        0.0  bending1        1          1  
4        0.0  bending1        1          1  


### 1. (b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.

In [3]:
test_data = data[((data['isBending'] == 1) & (data['dataset'] < 3)) | ((data['isBending'] == 0) & (data['dataset'] < 4))]

train_data = data[((data['isBending'] == 1) & (data['dataset'] > 2)) | ((data['isBending'] == 0) & (data['dataset'] > 3))]

### 1. (c) Feature Extraction: Classification of time series usually needs extracting features from them. In thisproblem, we focus on time-domain features.
#### i.  Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).


Time-domain features usually used in time series classification are: 
mean, standard deviation, maximum/minimum values, median, skewness, kurtosis

### 1. (c) ii. Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly.

In [4]:
timeFeatures = pd.DataFrame()
idx = 0;
for f in folders:
    # open folder, 
    folderFiles = os.listdir(parentFolder + f)
    for c in folderFiles:
        filePath = parentFolder + f + "/" + c
        if((f == folders[1]) & (c == 'dataset4.csv')):
            df = pd.read_csv(filePath, skiprows = 5, sep = ' ', names = colNames, usecols = range(7))
        else:
            df = pd.read_csv(filePath, header = 4, usecols = colNames)
        dataDictionary = {}
        dataDictionary['instance'] = idx
        dataDictionary['folderFile'] = f+"/"+c
        for i in range(1, np.size(colNames)):
            col = df.iloc[:,i]
            colDetails = pd.DataFrame(col.describe()).transpose()
            dataDictionary['min'+ str(i)] = colDetails['min'][0]
            dataDictionary['max'+ str(i)] = colDetails["max"][0]
            dataDictionary['mean'+ str(i)] = colDetails["mean"][0]
            dataDictionary['median'+ str(i)] = colDetails["50%"][0]
            dataDictionary['std'+ str(i)] = colDetails["std"][0]
            dataDictionary['1stQ'+ str(i)] = colDetails["25%"][0]
            dataDictionary['3rdQ'+ str(i)] = colDetails["75%"][0]
        timeFeatures = timeFeatures.append(dataDictionary, ignore_index=True)
        idx += 1
        
timeFeatures = timeFeatures.set_index('instance')
display(timeFeatures)




,folderFile,min1,max1,mean1,median1,std1,1stQ1,3rdQ1,min2,max2,...,std5,1stQ5,3rdQ5,min6,max6,mean6,median6,std6,1stQ6,3rdQ6
instance,,,,,,,,,,,,,,,,,,,,,
0,bending1/dataset1.csv,37.25,45.00,40.624792,40.500,1.476967,39.25,42.0000,0.0,1.30,...,2.188449,33.0000,36.00,0.0,1.92,0.570583,0.430,0.582915,0.00,1.3000
1,bending1/dataset2.csv,38.00,45.67,42.812812,42.500,1.435550,42.00,43.6700,0.0,1.22,...,1.995255,32.0000,34.50,0.0,3.11,0.571083,0.430,0.601010,0.00,1.3000
2,bending1/dataset3.csv,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,...,1.999604,35.3625,36.50,0.0,1.79,0.493292,0.430,0.513506,0.00,0.9400
3,bending1/dataset4.csv,33.00,47.75,42.179813,43.500,3.670666,39.15,45.0000,0.0,3.00,...,3.849448,30.4575,36.33,0.0,2.18,0.613521,0.500,0.524317,0.00,1.0000
4,bending1/dataset5.csv,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,...,2.411026,28.4575,31.25,0.0,1.79,0.383292,0.430,0.389164,0.00,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,walking/dataset5.csv,20.75,46.25,34.763333,35.290,4.742208,31.67,38.2500,0.0,12.68,...,3.174681,14.2500,18.33,0.0,9.39,3.288271,3.270,1.647528,2.05,4.3050
84,walking/dataset6.csv,21.50,51.00,34.935813,35.500,4.645944,32.00,38.0625,0.0,12.21,...,3.192058,14.2375,18.25,0.0,10.21,3.280021,3.015,1.700918,2.12,4.5000
85,walking/dataset7.csv,18.33,47.67,34.333042,34.750,4.948770,31.25,38.0000,0.0,12.48,...,3.000493,13.7500,18.00,0.0,8.01,3.261583,2.980,1.617290,2.05,4.3200


### 1. (c) iii. Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootstrap confidence interval for the standard deviation of each feature.


In [5]:
featuresStd = pd.DataFrame({"std":timeFeatures.std()})
intervals = []
for i in range(1, np.shape(timeFeatures)[1]):
    col = timeFeatures.iloc[:,i]
    col = (col,)
    bootstrap_ci = bootstrap(col, np.std, confidence_level= 0.9, method='basic')
    intervals.append(bootstrap_ci.confidence_interval)
featuresStd["confidence_interval"] = intervals

display(featuresStd)


,std,confidence_interval
min1,9.569975,"(8.318555536621998, 10.814373848606955)"
max1,4.394362,"(3.4816988497512797, 5.42120477934419)"
mean1,5.335718,"(4.765067184494108, 5.9345601964665144)"
median1,5.440054,"(4.8668629262891505, 6.064066632510744)"
std1,1.772153,"(1.5847008600722534, 1.9612584446449188)"
1stQ1,6.153590,"(5.628605839651356, 6.688002093097247)"
3rdQ1,5.138925,"(4.412127725187815, 5.914726869001741)"
min2,0.000000,"(0.0, 0.0)"
max2,5.062729,"(4.693767290274672, 5.469991764261947)"
mean2,1.574164,"(1.4386113380509131, 1.7384592613354652)"


### 1. (c) iv. Use your judgement to select the three most important time-domain features (one option may be min, mean, and max).


I believe mean, standard deviation and median are the three most important time-domain features. 

## 2. ISLR 3.7.4


(a) Even though the true relationship is linear, a higher degree polynomial fit in training would allow for a closer fit to the training data (which would most likely be an overfit), so I would expect the RSS for the cubic regression to be lower.

(b) For test data, I would expect the linear regression to be lower for the same reason. As mentioned above, the cubic regression would result in an overfit so the test data would perform poorly compared to the linear regression.

(c) For training data, we would expect the cubic regression to have a lower RSS, as it will be more flexible with more degrees of freedom and allow for a closer fit to the training data.

(d) For test data, there is not enough information to tell. If the true relationship is more similar to linear than it is to cubic, then the linear regression would perform better resulting in a lower RSS. If the inverse is true, then I would expect the cubic regression to perform better and have a lower RSS.

References: <br>
https://dataindependent.com/pandas/add-column-to-dataframe-pandas/ <br>
https://stackoverflow.com/questions/48042483/extra-commas-at-beginning-and-end-of-csv-rows-how-to-remove <br>
https://stats.stackexchange.com/questions/50807/features-for-time-series-classification <br>
https://www.digitalocean.com/community/tutorials/python-concatenate-string-and-int <br>
https://www.statology.org/bootstrapping-in-python/ <br>
